# Getting started with threads !!

Finding the current thread name

In [3]:
import threading
print(threading.current_thread().getName())
if threading.current_thread() == threading.main_thread():
    print("Current thread is main thread")

MainThread
Current thread is main thread


# Creating threads

Creating thread without using class

In [5]:
from threading import *
def display(val):
    print("In display function", val)
    
for i in range(5):
    t = Thread(target = display, args = (i, ))
    t.start()

In display functionIn display function 1
 0
In display function In display function 3
In display function 4
2


Creating thread using subclass of Thread class

In [9]:
from threading import *
class Mythread(Thread):
    def run(self):
        for i in range(5):
            print(i)

t = Mythread()
t2 = Mythread()
t.start()
t2.start()

0
1
2
3
4
0
1
2
3
4


Creating thread without subclass of Thread class

In [11]:
from threading import *
class MyClass:
    def __init__(self, value):
        self.i = value
    def display(self):
        print(self.i)
        print(threading.current_thread().getName())
    
obj = MyClass(1)
t1 = Thread(target = obj.display)
t1.setName("t1")
t2 = Thread(target = obj.display)
t2.setName("t2")
t3 = Thread(target = obj.display)
t3.setName("t3")
t1.start()
# Using join statements ensure one thread finishes execution before the second one starts \n",
# t1.join()\n",
t2.start()
# t2.join()\n",
t3.start()

1
t1
11
t3
t2



Single tasking using thread

In [15]:
from threading import *
from time import *
class ComplexTask:
    def execute(self):
        self.task1()
        self.task2()
        self.task3()
    def task1(self):
        print("Completing task 1")
        sleep(5)
    def task2(self):
        print("Completing task 2")
        sleep(10)
    def task3(self):
        print("Completing task 3")
        
obj = ComplexTask()
t = Thread(target = obj.execute)
t.start()

Completing task 1
Completing task 2
Completing task 3


Multitasking with threads

In [16]:
from threading import*
from time import *
class MovieHall:
    def __init__(self, name):
        self.name = name
    def display(self):
        for i in range(1, 5):
            print(self.name, i)
            sleep(1)
            
ob = MovieHall("Cutting ticket")
ob2 = MovieHall("Finding chair")
t1 = Thread(target = ob.display)
t2 = Thread(target = ob2.display)
t1.start()
t2.start()

Cutting ticketFinding chair 1 
1
Finding chairCutting ticket 2
 2
Cutting ticketFinding chair 3
 3
Cutting ticketFinding chair 4
 4


Two threads working on same method of same object causing inconsistency

In [19]:
from threading import *
class Railway:
    def __init__(self, vacant):
        self.vacant  = vacant
        
    def reserve(self, want):
        if self.vacant >= want:
            print(threading.current_thread(), "You are successful in reserving berth")
            # when sleep statement is executed we see in that time debo thread enters and books a berth
            sleep(1.5)
            self.vacant -= want
#             sleep(1.5)
            
        else:
            print(threading.current_thread(), "Sorry there's no berth available")
            
obj = Railway(1)
t1 = Thread(target = obj.reserve, args = (1, ))
t2 = Thread(target = obj.reserve, args = (1, ))
t1.setName("Sas")
t2.setName("Debo")
t1.start()
t2.start()
# One way to stop this is sleep after changing vacant but still that does not ensure safety from this inconsistency   

<Thread(Sas, started 139933826135808)> You are successful in reserving berth
<Thread(Debo, started 139933834528512)> You are successful in reserving berth


# Thread Synchronization

Using locks

In [20]:
from threading import *
class Railway:
    def __init__(self, vacant):
        self.vacant  = vacant
        self.l = Lock()
        
    def reserve(self, want):
        self.l.acquire()
        if self.vacant >= want:
            print(threading.current_thread(), "You are successful in reserving berth")
            # when sleep statement is executed we see in that time debo thread enters and books a berth
            sleep(1.5)
            self.vacant -= want
#              sleep(1.5)
            
        else:
            print(threading.current_thread(), "Sorry there's no berth available")
        self.l.release()
            
obj = Railway(1)
t1 = Thread(target = obj.reserve, args = (1, ))
t2 = Thread(target = obj.reserve, args = (1, ))
t1.setName("Sas")
t2.setName("Debo")
t1.start()
t2.start()

<Thread(Sas, started 139933834528512)> You are successful in reserving berth
<Thread(Debo, started 139933826135808)> Sorry there's no berth available


Using semaphore

In [22]:
from threading import *
class Railway:
    def __init__(self, vacant):
        self.vacant  = vacant
        self.l = Semaphore()
        
    def reserve(self, want):
        self.l.acquire()
        if self.vacant >= want:
            print(threading.current_thread(), "You are successful in reserving berth")
            # when sleep statement is executed we see in that time debo thread enters and books a berth
            sleep(1.5)
            self.vacant -= want
#              sleep(1.5)
            
        else:
            print(threading.current_thread(), "Sorry there's no berth available")
        self.l.release()
            
obj = Railway(1)
t1 = Thread(target = obj.reserve, args = (1, ))
t2 = Thread(target = obj.reserve, args = (1, ))
t1.setName("Sas")
t2.setName("Debo")
t1.start()
t2.start()
# If you want to allow more than threads to enter you can initialise self.l = Seaphore(counter)
# where counter is the number of threads you want to allow together

<Thread(Sas, started 139933834528512)> You are successful in reserving berth
<Thread(Debo, started 139933826135808)> Sorry there's no berth available


How to prevent deadlock of threads

In [26]:
class Railway:
    def __init__(self, vacant):
        self.vacant  = vacant
        self.l1 = Lock()
        self.l2 = Lock()
        
    def reserve(self, want):
        self.l1.acquire()
        print("Lock 1 acquired")
        self.l2.acquire()
        print("Lock 2 acquired")
        if self.vacant >= want:
            print(threading.current_thread(), "You are successful in reserving berth")
            # when sleep statement is executed we see in that time debo thread enters and books a berth
            sleep(1.5)
            self.vacant -= want
    #             sleep(1.5)

        else:
            print(threading.current_thread(), "Sorry there's no berth available")
        self.l2.release()
        print("Lock 2 released")
        self.l1.release()
        print("Lock 1 released")
        
    def cancel(self, delete):
        self.l1.acquire()
        print("Lock1 acquired")
        self.l2.acquire()
        print("Lock 2 acquired")
        self.vacant += delete
        print("Cancelling ticket successful")
        self.l2.release()
        print("Lock 2 released")
        self.l1.release()
        print("Lock 1 released")
        
obj = Railway(1)
t1 = Thread(target = obj.reserve, args = (1, ))
t2 = Thread(target = obj.cancel, args = (1, ))
t1.setName("Book ticket")
t2.setName("Cancel ticket")
t1.start()
t2.start()
# if lock 2 is acquired in cancel ticket first and then lock1 then deadlock may happen"

Lock 1 acquired
Lock 2 acquired
<Thread(Book ticket, started 139933826135808)> You are successful in reserving berth
Lock 2 released
Lock 1 releasedLock1 acquired
Lock 2 acquired
Cancelling ticket successful
Lock 2 released
Lock 1 released



# Communication between Threads

Using boolean type variable

In [27]:
class Producer:
    def __init__(self):
        self.producing = True
        self.list = []
        
    def produce(self):
        if self.producing:
            print("Production started")
            sleep(2)
            for i in range(10):
                self.list.append(i + 1)
                
        self.producing = False
        
class Consumer:
    def __init__(self, p):
        self.prod = p
        
    def consume(self):
        # if following lines are not written consumer starts consuming even before producer has started producing \n",
        # and prints empty list\n",
        while self.prod.producing :
            sleep(0.1)
        print("Consumption started")
        print(self.prod.list)
        
prod = Producer()
cons = Consumer(prod)
t1 = Thread(target = prod.produce)
t2 = Thread(target = cons.consume)
t1.start()
t2.start()
t1.join()
print(prod.list)

Production started
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Consumption started
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


Using notify and wait methods

In [28]:
class Producer:
    def __init__(self):
        self.cv = Condition()
        self.list = []
        
    def produce(self):
        self.cv.acquire()
        print("Production started")
        sleep(2)
        for i in range(10):
            self.list.append(i + 1)

        self.cv.notify()
        self.cv.release()
        
class Consumer:
    def __init__(self, p):
        self.prod = p
        
    def consume(self):
        self.prod.cv.acquire()
        print("Consumption started")
        print(self.prod.list)
        self.prod.cv.notify()
        self.prod.cv.release()

prod = Producer()
cons = Consumer(prod)
t1 = Thread(target = prod.produce)
t2 = Thread(target = cons.consume)
t1.start()
t2.start()

Production started
Consumption started
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


Using queue

In [30]:
from queue import *
class Producer:
    def __init__(self):
        
        self.queue = Queue()
        
    def produce(self):
        
        print("Production started")
        
        for i in range(10):
            self.queue.put(i + 1)
            sleep(1)
        
class Consumer:
    def __init__(self, p):
        self.prod = p
        
    def consume(self):
        print("Consumption started")
        for i in range(10):
            print(self.prod.queue.get())
        
prod = Producer()
cons = Consumer(prod)
t1 = Thread(target = prod.produce)
t2 = Thread(target = cons.consume)
t1.start()
t2.start()

Production started
Consumption started
1
2
3
4
5
6
7
8
9
10
